# Scraping: récupération de la page d'accueil du journal Le Soir

Dans ce notebook, nous créons un robot qui va ouvrir la page d'accueil du site du journal [Le Soir](https://www.lesoir.be/) et récupérer le titre de tous les articles du jour et les stocker dans un fichier csv.

## Imports

In [3]:
import os
import re
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Récupération de tous les articles de la page d'accueil



In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

articles = []

root_url = f"https://www.rtbf.be"
response = requests.get(root_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
for link in soup.find_all("a", {"class":"stretched-link outline-hidden"}):
    title = link.text.strip()
    url = root_url + link.get('href')
    articles.append([url, title])

In [5]:
# Affichage du nombre d'articles récupérés
len(articles)

80

In [6]:
# Affichage des 10 premières entrées
articles[:10]

[['https://www.rtbf.be/article/avortement-le-gouvernement-soumettra-une-proposition-de-loi-avant-les-vacances-11612870',
  'Avortement\xa0: le gouvernement soumettra une proposition de loi avant les vacances'],
 ['https://www.rtbf.be/article/amazon-va-livrer-le-jour-meme-l-impatience-est-elle-le-nouveau-reflexe-des-belges-11612706',
  'Amazon va livrer le jour même\xa0: l’impatience est-elle le nouveau réflexe des Belges\xa0?'],
 ['https://www.rtbf.be/article/la-crise-politique-en-france-une-serie-pas-tres-subtile-11612773',
  'La crise politique en France, une série pas très subtile'],
 ['https://www.rtbf.be/article/quand-le-handicap-ne-se-voit-pas-11612875',
  'Quand le handicap ne se voit pas'],
 ['https://www.rtbf.be/article/en-egypte-une-stele-antique-de-plus-de-4000-ans-a-mysterieusement-disparu-11612174',
  'En Égypte, une stèle antique de plus de\xa04000 ans a mystérieusement disparu'],
 ['https://www.rtbf.be/article/un-nombre-exceptionnel-de-puffins-fuligineux-observes-a-la-co

## Création d'un dataframe avec les liens et les titres des articles


In [7]:
df = pd.DataFrame(articles, columns=['link', 'title'])
df

,link,title
0,https://www.rtbf.be/article/avortement-le-gouv...,Avortement : le gouvernement soumettra une pro...
1,https://www.rtbf.be/article/amazon-va-livrer-l...,Amazon va livrer le jour même : l’impatience e...
2,https://www.rtbf.be/article/la-crise-politique...,"La crise politique en France, une série pas tr..."
3,https://www.rtbf.be/article/quand-le-handicap-...,Quand le handicap ne se voit pas
4,https://www.rtbf.be/article/en-egypte-une-stel...,"En Égypte, une stèle antique de plus de 4000 a..."
...,...,...
75,https://www.rtbf.behttps://auvio.rtbf.be/media...,Feux de l'amour S27
76,https://www.rtbf.behttps://auvio.rtbf.be/media...,Les Invisibles S04
77,https://www.rtbf.behttps://auvio.rtbf.be/media...,Enquête sur la crise en psychiatrie
78,https://www.rtbf.behttps://auvio.rtbf.be/media...,Une affaire d'honneur


In [8]:
# Sauvegarde du dataframe dans un fichier csv
df.to_csv(f"../data/rtbf_{time.strftime('%Y%m%d')}.csv", index=False)

## Téléchargement d'un article et affichage du texte

In [9]:
# Récupération du premier lien du dataframe
article_url = df.iloc[0][0]

# Ouverture de la page
response = requests.get(article_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
# Parsing du contenu
title = soup.find("h1").get_text(separator=' ')

paragraphs = []
for paragraph in soup.find_all("p", attrs={"class": None}):
    paragraphs.append(paragraph.get_text(separator=' ').strip())

content = "\n".join(paragraphs)
print(title)
print("==================================")
print(content)

C:\Users\Ing Armel Fopa\AppData\Local\Temp\ipykernel_24888\3125040403.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  article_url = df.iloc[0][0]


Avortement : le gouvernement soumettra une proposition de loi avant les vacances
En attendant, la commission de la Justice de la Chambre a rejeté une proposition de loi d’Ecolo-Groen qui dépénalise le droit à l’avortement, supprime le délai de réflexion et  allonge le recours à la procédure à 18 semaines .
La séance a connu un retard au démarrage en l’absence de la majorité. La coalition Arizona est divisée sur cette question entre partis qui soutiennent le dispositif proposé par une partie de l’opposition, et qui s’appuie sur un rapport d’experts pluridisciplinaires, et les partis qui y sont opposés, en particulier le  CD&V  et la N-VA.
" L’accord de coalition est clair : la législation sera modifiée. La ministre de la Justice Annelies Verlinden travaille actuellement sur un texte ", a assuré Mme Farih. La députée a rappelé que d’autres questions éthiques étaient également à l’ordre du jour, telles que la maternité de substitution, l’accouchement discret, l’euthanasie en cas de démenc

### Nettoyage du texte à l'aide d'expressions régulières

In [10]:
# Suppression de tout ce qui se trouve entre parenthèses
clean_content = re.sub("\([^\)]+\)", " ", content)
# Suppression des espaces multiples
clean_content = re.sub("\s+", " ", clean_content)

print(content)
print("==================================")
print(clean_content)


En attendant, la commission de la Justice de la Chambre a rejeté une proposition de loi d’Ecolo-Groen qui dépénalise le droit à l’avortement, supprime le délai de réflexion et  allonge le recours à la procédure à 18 semaines .
La séance a connu un retard au démarrage en l’absence de la majorité. La coalition Arizona est divisée sur cette question entre partis qui soutiennent le dispositif proposé par une partie de l’opposition, et qui s’appuie sur un rapport d’experts pluridisciplinaires, et les partis qui y sont opposés, en particulier le  CD&V  et la N-VA.
" L’accord de coalition est clair : la législation sera modifiée. La ministre de la Justice Annelies Verlinden travaille actuellement sur un texte ", a assuré Mme Farih. La députée a rappelé que d’autres questions éthiques étaient également à l’ordre du jour, telles que la maternité de substitution, l’accouchement discret, l’euthanasie en cas de démence, etc. L’argument n’a pas convaincu l’opposition, Vlaams Belang excepté.
L’autri

<>:2: SyntaxWarning: invalid escape sequence '\('
<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\('
<>:4: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Ing Armel Fopa\AppData\Local\Temp\ipykernel_24888\3799268790.py:2: SyntaxWarning: invalid escape sequence '\('
  clean_content = re.sub("\([^\)]+\)", " ", content)
C:\Users\Ing Armel Fopa\AppData\Local\Temp\ipykernel_24888\3799268790.py:4: SyntaxWarning: invalid escape sequence '\s'
  clean_content = re.sub("\s+", " ", clean_content)


### Création d'un fichier avec le contenu de l'article


In [11]:
with open("../data/rtbf_example.txt", "w") as writer:
    writer.write(f"{title}\n\n{clean_content}")

## Pour en savoir plus

- Le web scraping avec Python: https://realpython.com/beautiful-soup-web-scraper-python/
- Tutoriel sur les expressions régulières: https://www.w3schools.com/python/python_regex.asp